In [22]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.applications as apps
import pandas as pd
import kagglehub
from pathlib import Path
import numpy as np
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split
import gc


In [2]:
def set_memory_growth():
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
      try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
          tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
      except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

def set_memory_limit(memory_limit):
    gpus = tf.config.list_physical_devices('GPU')
    if gpus:
        tf.config.set_logical_device_configuration(
            gpus[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=memory_limit)]
        )

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")

#set_memory_limit(4096)
set_memory_growth()
keras.mixed_precision.set_global_policy("mixed_float16")

pd.set_option('display.max_columns', None)

1 Physical GPU, 1 Logical GPUs


I0000 00:00:1744116895.965145   34807 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4096 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [3]:
SEED = 42
BATCH_SIZE = 8
PATIENCE = 5

saved_models_dir = Path("../saved_models")
saved_models_dir.mkdir(parents=True, exist_ok=True)
histories_dir = Path("../histories")
histories_dir.mkdir(parents=True, exist_ok=True)

# Create Datasets

In [4]:
# Download latest version of data
image_dir = kagglehub.dataset_download("bloox2/fieldplant")
image_dir = Path(image_dir) / "train"
print("Path to dataset files:", image_dir)

Path to dataset files: /home/ruairi/.cache/kagglehub/datasets/bloox2/fieldplant/versions/1/train


# Create Models

In [5]:
def get_model_and_preprocessing(app_name):
    app = getattr(keras.applications, app_name)
    model_name = dir(app)[0]
    model = getattr(app, model_name)
    input_shape = model().input_shape[1:]
    model = model(include_top=False, input_shape=input_shape)
    model.trainable = False
    preprocessing = getattr(app, "preprocess_input")
    return model, preprocessing

In [6]:
def get_pre_classifier_layers(model_name):
    if "vgg" in model_name:
        return keras.Sequential([
            keras.layers.GlobalAveragePooling2D(),
            keras.layers.Dense(1024, activation="relu"),
            keras.layers.Dense(1024, activation="relu") ])
    else:
        return keras.Sequential([keras.layers.GlobalAveragePooling2D()])

In [7]:
def build_model(app_name, activation, num_classes):
    model, preprocessing = get_model_and_preprocessing(app_name)
    pre_classifier_layers = get_pre_classifier_layers(model.name)

    inputs = keras.Input(shape=model.input_shape[1:])
    x = preprocessing(inputs)
    x = model(x, training=False)
    x = pre_classifier_layers(x)
    outputs = keras.layers.Dense(num_classes, activation=activation, name="classifier_layer")(x)

    model_name = model.name
    model = keras.Model(inputs, outputs, name=model_name)

    return model

In [8]:
def get_hyperparameters(methodology):
    multilabel = methodology == "multilabel"
    
    methodologies = ["multiclass", "multilabel"]
    losses = ["categorical_crossentropy", "binary_crossentropy"]
    activation = ["softmax", "sigmoid"]
    metrics = ["categorical_accuracy", "binary_accuracy"]

    idx = methodologies.index(methodology)

    metrics = [metrics[idx]]
    f1_score_weighted = keras.metrics.F1Score(average="weighted", threshold=0.5 if multilabel else None, name="f1_score_weighted", dtype=None)
    f1_score_per_class = keras.metrics.F1Score(average=None, threshold=0.5 if multilabel else None, name="f1_score_per_class", dtype=None)
    metrics.append(f1_score_weighted)
    metrics.append(f1_score_per_class)
        
    hyperparams = [losses[idx], activation[idx], metrics]
    
    return hyperparams

In [9]:
def print_model_info(model):
    compile_config = model._compile_config.config
    optimizer = compile_config['optimizer'].get_config()
    classifier_activation = model.get_layer(name="classifier_layer").activation.__name__

    print("Model name:", model.name)
    print("Input shape:", model.input_shape)
    print("Optimizer name:", optimizer['name'], "learning_rate:", np.round(optimizer['learning_rate'], 6))
    print("Loss:", compile_config['loss'])
    print("Metrics:")
    for metric in compile_config['metrics']:
        print(metric if isinstance(metric, str) else metric.get_config())
    print("Classifier layer activation function:", classifier_activation)
    print()

In [10]:
def get_callbacks(model_name, fine_tuning=False):
    cbs = [
        keras.callbacks.EarlyStopping(patience=PATIENCE, restore_best_weights=True, baseline=None, verbose=1),
        keras.callbacks.ModelCheckpoint(filepath=f"{saved_models_dir}/{model_name}.keras", save_best_only=True, monitor="val_loss", verbose=1, initial_value_threshold=None)]
    return cbs


# GET DATASETS

In [11]:
def get_dataframe(filtered=False, sample=False):
    filename = "filtered" if filtered else "unfiltered"
    all_csv_files = list(Path("../data").glob("*"))
    csv_file = [csv for csv in all_csv_files if filename in csv.name][0]
    df = pd.read_csv(csv_file)
    print()

    if sample:
        print("Using sampled DF")
        df = df.sample(frac=0.2)
        df = df.loc[(df!=0).any(axis=1)]
    print("df shape:", df.shape)
    return df   

In [12]:
def get_train_test_splits(df, filtered=False, test_size=0.2):
    col_names = list(df.columns)
    split_fn = get_stratified_splits if filtered else get_nonstratified_splits
    (X_train, X_test, X_val, y_train, y_test, y_val) = split_fn(df, test_size=test_size)
    train_df = pd.merge(X_train, y_train, left_index=True, right_index=True)
    test_df = pd.merge(X_test, y_test, left_index=True, right_index=True)
    val_df = pd.merge(X_val, y_val, left_index=True, right_index=True)
    
    train_df.columns = col_names
    test_df.columns = col_names
    val_df.columns = col_names

    return train_df, test_df, val_df
    
def get_stratified_splits(df, test_size=0.2):
    columns = list(df.columns)
    X = df.filename.to_frame().to_numpy()
    y = df.drop(columns=["filename"]).to_numpy()

    X_train, y_train, X_test_val, y_test_val = iterative_train_test_split(X, y, test_size=0.2)
    X_test, y_test, X_val, y_val = iterative_train_test_split(X_test_val, y_test_val, test_size=0.5)
    datasets = (X_train, X_test, X_val, y_train, y_test, y_val)
    datasets = [pd.DataFrame(dataset) for dataset in datasets]
    return tuple(datasets)

def get_nonstratified_splits(df, test_size=0.2):
    X = df.filename
    y = df.drop(columns=["filename"])
    X_train, X_test_val, y_train, y_test_val = train_test_split(X, y, test_size=0.2, random_state=SEED)
    X_test, X_val, y_test, y_val = train_test_split(X_test_val, y_test_val, test_size=0.5, random_state=SEED)

    return (X_train, X_test, X_val, y_train, y_test, y_val)

In [13]:
def decode_img(filename, img_size):
    filepath = str(image_dir) + "/" + filename
    img = tf.io.read_file(filepath)
    img = tf.io.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, img_size)
    return img
    
def process_dataset(filename, labels, img_size):
    img = decode_img(filename, img_size=img_size)
    return img, labels

def configure_datasets_for_performance(datasets, shuffle=False, batch_size=BATCH_SIZE):
    configured_datasets = []
    for dataset in datasets:
        if shuffle:
            dataset = dataset.shuffle(buffer_size=dataset.cardinality(), reshuffle_each_iteration=True)
        dataset = dataset.batch(batch_size=batch_size, num_parallel_calls=tf.data.AUTOTUNE)
        dataset = dataset.cache()
        dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
        configured_datasets.append(dataset)
    return tuple(configured_datasets)

In [14]:
def datasets_from_dataframes(img_size, splits=None):
    datasets = []
    for split in splits:
        img = split.filename
        labels = split.drop(columns=["filename"])
        dataset = tf.data.Dataset.from_tensor_slices((img, labels))
        dataset = dataset.map(lambda x,y: process_dataset(x,y, img_size))
        datasets.append(dataset)
    return tuple(datasets)

In [15]:
def get_datasets(img_size, df, test_size=0.2):
    splits = get_train_test_splits(df, test_size=test_size)
    datasets = datasets_from_dataframes(img_size, splits=splits)
    for dataset in zip(["train", "val", "test"], datasets):
        print(dataset[0],"size:", len(dataset[1]))
    return datasets

# CREATE MODELS

In [16]:
def get_freeze_at_layer(model_name):
    model_name = model_name.lower()
    if "mobilenet" in model_name:
        # 120 used in FP
        # last block 143
        return 120
    if "vgg16" in model_name:
        # 14 used in FP
        # last layer 17
        return 14
    if "inceptionv3" in model_name:
        # 172 used in FP
        # last block 279
        return 172
    if "inceptionresnetv2" in model_name:
        # 516 used in FP
        # 761 last block
        return 516

    # Function to freeze the given base model at the desired layer number
def freeze_model(base_model, freeze_at):
    print(f"Unfreezing from {freeze_at}.")
    # freezes initial layers
    for layer in base_model.layers[:freeze_at]:
        layer.trainable=False
    # unfreezes later
    for layer in base_model.layers[freeze_at:]:
        layer.trainable = True

In [17]:
def compile_model(model, loss=None, metrics=[], fine_tuning=False):
    opt = keras.optimizers.Adam
    lr = (float(opt().learning_rate) / 10) if fine_tuning else float(opt().learning_rate)
    
    model.compile(
        loss=loss,
        optimizer=keras.optimizers.Adam(learning_rate=lr, gradient_accumulation_steps=4),
        metrics=metrics
    )
    print_model_info(model)

In [18]:
def train_model(model=None, train_ds=None, val_ds=None, epochs=100, loss=None, metrics=[], fine_tuning=False, initial_epoch=0):
    if fine_tuning:
        freeze_layer = get_freeze_at_layer(model.name)
        freeze_model(model, freeze_layer)
        
    compile_model(model, loss=loss, metrics=metrics, fine_tuning=fine_tuning)
    
    string_suffix = "FT" if fine_tuning else "CLF"
    print("Training -", string_suffix)
   
    cbs = get_callbacks(model.name, fine_tuning=fine_tuning)

    print(f"Starting training at epoch {initial_epoch}." )
    history = model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=cbs, initial_epoch=initial_epoch)
    history_df = pd.DataFrame(history.history)
    history_df['model'] = model.name
    history_df['epoch'] = history.epoch
    history_df['type'] = string_suffix
    display(history_df)
    history_df.to_csv(str(histories_dir) + "/" + model.name + string_suffix + ".csv", index=False)  
    
    best_epoch = history_df.loc[history_df.val_loss == history_df.val_loss.min(), "epoch"].values[0] + 1
    print(f"Best epoch number: {best_epoch}")
    return best_epoch
    

In [19]:
def run_all(epochs=5, sample=True):
    i = 0
    num_models = 0

   # app_names = ["mobilenet_v2", "vgg16", "inception_v3", "inception_resnet_v2"]
    app_names = ["inception_resnet_v2"]
    
  #  methodologies = ["multiclass", "multilabel"]
    methodologies = ["multiclass"]

   # filter_options = [False, True]
    filter_options = [True]


    # Get total count of models to be training
    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
            for app_name in app_names:
                num_models += 1

    # Training Loop
    for filtered in filter_options:
        for methodology in methodologies:
            if not filtered and (methodology == "multilabel"):
                continue
                
            loss, activation, metrics = get_hyperparameters(methodology)
    
            df = get_dataframe(filtered=filtered, sample=sample)
            num_classes = len(df.columns[1:])
            
            for app_name in app_names:
                i += 1
                print()
                print(f"Model: {i} of {num_models}")
                print("Filtered dataset:", filtered)
                print("Methodology:", methodology)
       
                model = build_model(app_name, activation=activation, num_classes=num_classes)

                img_size = model.input_shape[1:3]
                
                datasets = get_datasets(img_size, df=df)
                train_ds, test_ds, val_ds = configure_datasets_for_performance(datasets)

                model.name = model.name + "_" + ("filtered" if filtered else "unfiltered") + "_" + methodology
                print("Save file name:", model.name)

                # Train top Classifier
                best_epoch = train_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=False, initial_epoch=0)
                # Fine Tuning
                ft_start_epoch = best_epoch + 1
                train_model(model=model, train_ds=train_ds, val_ds=val_ds, epochs=epochs, loss=loss, metrics=metrics, fine_tuning=True, initial_epoch=ft_start_epoch)

                del model
                gc.collect()
                tensorflow.keras.backend.clear_session()

In [20]:
model = run_all(epochs=10, sample=False)


df shape: (5154, 28)

Model: 1 of 1
Filtered dataset: True
Methodology: multiclass
train size: 4123
val size: 515
test size: 516
Save file name: inception_resnet_v2_filtered_multiclass
Model name: inception_resnet_v2_filtered_multiclass
Input shape: (None, 299, 299, 3)
Optimizer name: adam learning_rate: 0.001
Loss: categorical_crossentropy
Metrics:
categorical_accuracy
{'name': 'f1_score_weighted', 'dtype': 'float32', 'average': 'weighted', 'threshold': None}
{'name': 'f1_score_per_class', 'dtype': 'float32', 'average': None, 'threshold': None}
Classifier layer activation function: softmax

Training - CLF
Starting training at epoch 0.
Epoch 1/100


I0000 00:00:1744116915.919498   34917 service.cc:152] XLA service 0x7fb2e00b0380 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1744116915.919568   34917 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2025-04-08 13:55:16.918849: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1744116920.070982   34917 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1744116931.257708   34917 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


516/516 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - categorical_accuracy: 0.4916 - f1_score_per_class: 0.0981 - f1_score_weighted: 0.4081 - loss: 1.9857

2025-04-08 13:56:20.387009: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6134', 180 bytes spill stores, 180 bytes spill loads

2025-04-08 13:56:29.452302: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_6134', 180 bytes spill stores, 180 bytes spill loads




Epoch 1: val_loss improved from inf to 1.14157, saving model to ../saved_models/inception_resnet_v2_filtered_multiclass.keras
516/516 ━━━━━━━━━━━━━━━━━━━━ 97s 135ms/step - categorical_accuracy: 0.4918 - f1_score_per_class: 0.0982 - f1_score_weighted: 0.4083 - loss: 1.9849 - val_categorical_accuracy: 0.6725 - val_f1_score_per_class: 0.1830 - val_f1_score_weighted: 0.5942 - val_loss: 1.1416
Epoch 2/100
515/516 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - categorical_accuracy: 0.6943 - f1_score_per_class: 0.2579 - f1_score_weighted: 0.6364 - loss: 1.0254
Epoch 2: val_loss improved from 1.14157 to 0.99607, saving model to ../saved_models/inception_resnet_v2_filtered_multiclass.keras
516/516 ━━━━━━━━━━━━━━━━━━━━ 29s 55ms/step - categorical_accuracy: 0.6943 - f1_score_per_class: 0.2580 - f1_score_weighted: 0.6364 - loss: 1.0253 - val_categorical_accuracy: 0.6996 - val_f1_score_per_class: 0.2532 - val_f1_score_weighted: 0.6454 - val_loss: 0.9961
Epoch 3/100
234/516 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step 

KeyboardInterrupt: 